# 卷积神经网络

## 引入环境

In [3]:
import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms

# 接入设备
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

## 获得数据

In [7]:
batch_size = 100

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../../../datasets/',
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

test_dataset = torchvision.datasets.MNIST(root='../../../datasets/',
                                          train=False, 
                                          transform=transforms.ToTensor())

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size, 
                                          shuffle=False)

## 搭建卷积神经网络

In [12]:
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        self.layer1 = nn.Sequential(
                        nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
                        nn.BatchNorm2d(16),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.layer2 = nn.Sequential(
                        nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
                        nn.BatchNorm2d(32),
                        nn.ReLU(),
                        nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)
    
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.reshape(out.size(0), -1)
        out = self.fc(out)
        return out

## 训练模型

In [14]:
# 超参数 
num_classes = 10
learning_rate = 0.0001
num_epochs = 8
# 实例化模型
model = ConvNet(num_classes).to(device)
# 指定损失和优化
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)


total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # 前向传播
        outputs = model(images)
        loss = criterion(outputs, labels)

        # 反向传播+优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/8], Step [100/600], Loss0.7172
Epoch [1/8], Step [200/600], Loss0.5477
Epoch [1/8], Step [300/600], Loss0.2489
Epoch [1/8], Step [400/600], Loss0.1865
Epoch [1/8], Step [500/600], Loss0.2465
Epoch [1/8], Step [600/600], Loss0.1663
Epoch [2/8], Step [100/600], Loss0.1610
Epoch [2/8], Step [200/600], Loss0.2148
Epoch [2/8], Step [300/600], Loss0.0667
Epoch [2/8], Step [400/600], Loss0.0542
Epoch [2/8], Step [500/600], Loss0.0635
Epoch [2/8], Step [600/600], Loss0.0300
Epoch [3/8], Step [100/600], Loss0.0667
Epoch [3/8], Step [200/600], Loss0.0407
Epoch [3/8], Step [300/600], Loss0.0675
Epoch [3/8], Step [400/600], Loss0.0797
Epoch [3/8], Step [500/600], Loss0.0653
Epoch [3/8], Step [600/600], Loss0.0822
Epoch [4/8], Step [100/600], Loss0.0331
Epoch [4/8], Step [200/600], Loss0.0837
Epoch [4/8], Step [300/600], Loss0.0279
Epoch [4/8], Step [400/600], Loss0.0368
Epoch [4/8], Step [500/600], Loss0.0318
Epoch [4/8], Step [600/600], Loss0.0432
Epoch [5/8], Step [100/600], Loss0.0691


## 测试模型

In [15]:
# 将model的training参数设为False，让BatchNorm和Drop等层不起作用
model.eval()
# 通过no_grad()禁止梯度求导操作节省内存消耗
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
torch.save(model.state_dict(), '../../../datasets/convModel.ckpt')

Test Accuracy of the model on the 10000 test images: 98.66 %


## 测试保存的模型重新使用

In [16]:
anotherModel = ConvNet(num_classes).to(device)
anotherModel.load_state_dict(torch.load('../../../datasets/convModel.ckpt'))

# 将model的training参数设为False，让BatchNorm和Drop等层不起作用
anotherModel.eval()
# 通过no_grad()禁止梯度求导操作节省内存消耗
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = anotherModel(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    print('Test Accuracy of the model on the 10000 test images: {} %'.format(100 * correct / total))
torch.save(anotherModel.state_dict(), '../../../datasets/convModel.ckpt')

Test Accuracy of the model on the 10000 test images: 98.66 %
